# Proyecto final Inteligencia Computacional
## Style Transfer
## ENTRENAMIENTO
### [Ammi Beltrán Troppa, Fernanda Borja Muñoz]

# Imports

In [ ]:
import os
import io
import sys
import time
#
import numpy as np
import torch
from torch import nn
import torchvision
from matplotlib import pyplot as plt
#
import zipfile
#
from PIL import Image
from torch.utils.data import Dataset
from natsort import natsorted
from torchvision import datasets, transforms

# Encoder y Decoder

In [ ]:
class Encoder(nn.Module):
  def __init__(self,layer = 15):
    super().__init__()
    fully_model = torchvision.models.vgg16(weights = torchvision.models.vgg.VGG16_Weights.DEFAULT).eval().features
    self.model = fully_model[:layer + 1]

  def forward(self,x):
    return self.model(x)

In [ ]:
Encode = Encoder(layer = 15)
Encode

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Encoder(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
  )
)

In [ ]:
class Decoder(nn.Module):
  def __init__(self,layer = 14):
    super().__init__()
    fully_decoder = nn.Sequential(
        # Primera capa
        nn.Upsample(scale_factor=(2,2), mode='nearest'),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        
        # Segunda Capa
        nn.Upsample(scale_factor=(2,2), mode='nearest'),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),

        #Tercera Capa
        nn.Upsample(scale_factor=(2,2), mode='nearest'),
        nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        

        #Cuarta Capa
        nn.Upsample(scale_factor=(2,2), mode='nearest'),
        nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        

        #Quinta Capa
        nn.Upsample(scale_factor=(2,2), mode='nearest'),
        nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True),
        nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(inplace = True), 
    )
    self.model = fully_decoder[layer:29 ]

  def forward(self,x):
    return self.model(x)

In [ ]:
Decode = Decoder(layer = 14)
Decode

Decoder(
  (model): Sequential(
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace=True)
    (18): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (19): ReLU(inplace=True)
    (20): Upsample(scale_factor=(2.0, 2.0), mode=nearest)
    (21): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (22): ReLU(inplace=True)
    (23): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (24): ReLU(inplace=True)
    (25): Upsample(scale_factor=(2.0, 2.0), mode=nearest)
    (26): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (27): ReLU(inplace=True)
    (28): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

# Data 

## Colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
!cp /content/drive/MyDrive/data/val2017.zip /content/
!unzip -qo /content/val2017.zip -d /content/
!rm /content/val2017.zip

cp: cannot stat '/content/drive/MyDrive/data/val2014.zip': No such file or directory
unzip:  cannot find or open /content/val2014.zip, /content/val2014.zip.zip or /content/val2014.zip.ZIP.
rm: cannot remove '/content/val2014.zip': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/data/test2014.zip /content/
!unzip -qo /content/test2014.zip -d /content/
!rm /content/test2014.zip

Aprovecho de crear el array de entrenamiento

In [ ]:
dir = "/content/test2014"
obj = natsorted(os.listdir(dir))
# #Entonces puedo armar una lista con todas las direcciones
dirTrain = []
for i in range(0, len(obj)):
   dirTrain.append(os.path.join(dir, obj[i]))

In [ ]:
len(dirTrain)

40775

Y el de validacion

In [ ]:
dir = "/content/val2014"
obj = natsorted(os.listdir(dir))
# #Entonces puedo armar una lista con todas las direcciones
dirVal = []
for i in range(0, len(obj)):
   dirVal.append(os.path.join(dir, obj[i]))

In [ ]:
len(dirVal)

5000

# Dataset

Transformaciones

In [ ]:
#Transformación para imagenes 
transform1 = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])

Dataset Antiguo

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, imagelist, transform, encoder):
        super().__init__()
        self.imagepaths = imagelist
        self.transform = transform
        self.encoder = encoder

    def __len__(self):
        return len(self.imagepaths)

    def __getitem__(self, index):
        imagepath = self.imagepaths[index]
        image = Image.open(imagepath).convert('RGB')
        
        Norm_image = self.transform(image)
        feature = self.encoder(Norm_image.to('cuda'))
        
        return feature.cpu(), Norm_image.cpu()

In [ ]:
Data = CustomDataSet(dirTrain, transform = transform1, encoder = Encode)
Val = CustomDataSet(dirVal, transform = transform1, encoder = Encode)

# Código entrenamiento

In [ ]:
class DecoderLoss(nn.Module):
  def __init__(self,weight = 1):
    super().__init__()
    self.weight = weight
  
  def forward(self, ydif, fdif): 
    reconst = torch.norm(ydif)**2
    features = self.weight * torch.norm(fdif)**2
    return reconst + features
  

In [ ]:
def show_curves(curves):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(13, 5))
    fig.set_facecolor('white')

    epochs = np.arange(len(curves["val_loss"])) + 1

    ax2.plot(epochs, curves['train_loss'], label='Training',color='r')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Loss evolution during training (Training)')
    ax2.legend()


    ax1.plot(epochs, curves['val_loss'], label='Validation', color='b')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss evolution during training (Validation)')
    ax1.legend()

    plt.show()

In [ ]:
def train_step(x_batch, y_batch, model, optimizer, criterion, deco,use_gpu = False ):
    # Predicción
    y_predicted = model(x_batch)

    ydif = y_predicted - y_batch
    fdif = deco(y_predicted) - x_batch #deco(y_batch)

    # Cálculo de loss
    loss = criterion(ydif, fdif)

    # Actualización de parámetros
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return y_predicted, loss

def evaluate(val_loader, model, criterion, deco , use_gpu = False):
    cumulative_loss = 0

    for x_val, y_val in val_loader:

        if use_gpu:
            x_val = x_val.cuda()
            y_val = y_val.cuda()

        y_predicted = model(x_val)
        ydif = y_predicted - y_val
        fdif = deco(y_predicted) - x_val #deco(y_val)

        
        loss = criterion(ydif, fdif)

        cumulative_loss += loss.item()
        del x_val, y_val
    val_loss = cumulative_loss / len(val_loader)

    return val_loss


def train_model(
    model,
    train_dataset,
    val_dataset,
    epochs,
    criterion,
    batch_size,
    lr,
    n_evaluations_per_epoch=30,
    use_gpu=False,
    name = 'Deco',
    deco = Encode,
    perdida = 'loser',
    lastEpoch = 0
):
    deco.cpu()
    model.cpu()
    if use_gpu:
        model.cuda()
        Encode.cuda()

    # Definición de dataloader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=use_gpu)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=int(len(val_dataset)/20), shuffle=False, pin_memory=use_gpu) #batch_size=len(val_dataset)

    # Optimizador
    #optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Listas para guardar curvas de entrenamiento
    if  os.path.exists(perdida + ".pt"):
        curves = torch.load(f'{perdida}.pt')
    else:
        curves = {
            "train_loss": [],
            "val_loss": [],
        }

    t0 = time.perf_counter()

    iteration = 0

    n_batches = len(train_loader)

    best_valid_loss = float('inf')

    for epoch in range(epochs):
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        print(f"\rEpoch {epoch + lastEpoch + 1}/{epochs + lastEpoch}")
        cumulative_train_loss = 0
        train_loss_count = 0
       

        # Entrenamiento del modelo
        model.train()
        for i, (x_batch, y_batch) in enumerate(train_loader):
            if use_gpu:
                x_batch = x_batch.cuda()
                y_batch = y_batch.cuda()


            y_predicted, loss = train_step(x_batch, y_batch, model, optimizer, criterion, deco,use_gpu )

            cumulative_train_loss += loss.item()
            train_loss_count += 1


            if (i % (n_batches // n_evaluations_per_epoch) == 0) and (i > 0):
                train_loss = cumulative_train_loss / train_loss_count

                print(f"Iteration {iteration} - Batch {i}/{len(train_loader)} - Train loss: {train_loss}")

            iteration += 1
            del x_batch, y_batch

        model.eval()
        with torch.no_grad():
            val_loss = evaluate(val_loader, model, criterion,deco, use_gpu)

        #if val_loss < best_valid_loss:
        #   best_valid_loss = val_loss
        torch.save(model.state_dict(), f'{name}{epoch + lastEpoch + 1}.pt')

        print(f"Val loss: {val_loss}")

        train_loss = cumulative_train_loss / train_loss_count

        curves["train_loss"].append(train_loss)
        curves["val_loss"].append(val_loss)

        torch.save(curves, f'{perdida}.pt')

    print()
    print(f"Tiempo total de entrenamiento: {time.perf_counter() - t0:.4f} [s]")

    model.load_state_dict(torch.load(f'{name}{epoch + lastEpoch + 1}.pt'))

    return show_curves(curves)

# Entrenamiento

### Forma nueva

In [ ]:
Decode.load_state_dict(torch.load('3L-16.pt'))

<All keys matched successfully>

In [ ]:
epochs = 5
criterion = DecoderLoss()
batch_size = 45
lr = 1 * 10**(-3)
use_gpu = True
name = f'3L-' #Nombre de como queremos que se guarde el mejor modelo resultante de cada época
perdida = 'loss-3L'
lastEpoch = 16
train_model(
    model = Decode,
    train_dataset = Data, val_dataset = Val,
    epochs = epochs, 
    criterion = criterion,
    batch_size = batch_size, lr = lr,
    use_gpu = use_gpu,
    n_evaluations_per_epoch =20 , #334 #Cada cuantos batches te muestra el progreso de loss de entrenamiento
    name = name,
    perdida = perdida,
    lastEpoch = lastEpoch
    )

Epoch 17/21
Iteration 45 - Batch 45/904 - Train loss: 145040806.4347826
Iteration 90 - Batch 90/904 - Train loss: 115072689.23076923
Iteration 135 - Batch 135/904 - Train loss: 102772837.11764705
Iteration 180 - Batch 180/904 - Train loss: 96495074.6961326
Iteration 225 - Batch 225/904 - Train loss: 91885257.36283186
Iteration 270 - Batch 270/904 - Train loss: 89374969.88929889
Iteration 315 - Batch 315/904 - Train loss: 86807238.73417722
Iteration 360 - Batch 360/904 - Train loss: 84956622.27146815
Iteration 405 - Batch 405/904 - Train loss: 83477479.30049261
Iteration 450 - Batch 450/904 - Train loss: 82189091.654102
Iteration 495 - Batch 495/904 - Train loss: 81190390.41129032
Iteration 540 - Batch 540/904 - Train loss: 80413684.11090574
Iteration 585 - Batch 585/904 - Train loss: 79549324.0887372
Iteration 630 - Batch 630/904 - Train loss: 79042294.65610142
Iteration 675 - Batch 675/904 - Train loss: 78618821.05325444
Iteration 720 - Batch 720/904 - Train loss: 78178864.85436893
It